In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
warnings.filterwarnings('ignore')

In [2]:
# parameters
customer_id = '115'
formatted_attribute = 'Preferences'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

# buckets = """Timing Belt"""

regex_pattern = r"(?i)(max)(min)"
# regex_pattern = r"(?i)(\b[8][1-9] in\w* Overall Length)|(\b[9][0-9] in\w* Overall Length)|(\b1[0][0] in\w* Overall Length)"


value='%n/a%'
customer_name='%lovelyskin%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
#           'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [3]:
metric = ' Inches'

range_string = """
 0 - 0.5000 Inches
 0 - 10 Inches
 0.5001 - 1.0000 Inches
 0 - 5 Inches
 1.0001 - 1.5000 Inches
 11 - 15 Inches
 11 - 20 Inches
 1.5001 - 2.0000 Inches
 16 - 20 Inches
 2.0001 - 2.5000 Inches
 21 - 25 Inches
 21 - 30 Inches
 2.5001 - 3.0000 Inches
 26 - 30 Inches
 3.0001 - 3.5000 Inches
 31 - 40 Inches
 3.5001 - 4.0000 Inches
 4.0001 - 4.5000 Inches
 41 - 50 Inches
 4.5001 - 5.0000 Inches
 6 - 10 Inches
 Greater Than 30 Inches
 Greater than 5.0000 Inches
 Greater than 50 Inches
 """

In [4]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [5]:
df = query_from_file(file_name='./query/lovelySkinPullData.sql', params=params)
print(len(df))
df

0


Empty DataFrame
Columns: [bucket, attributes, external_id, name, description]
Index: []

In [13]:
df['attributes']=df['attributes'].str.get(0)
df

Empty DataFrame
Columns: [bucket, attributes, external_id, name, description]
Index: []

In [7]:
Concern=df[df['attributes'].astype(str)=='Concern']
print(len(Concern))
Concern

0


Empty DataFrame
Columns: [bucket, attributes, external_id, name, description]
Index: []

In [8]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

['Acne & Blemishes',
 'Anti-Aging',
 'Antibacterial',
 'Anti-Inflammatory',
 'Blackheads',
 'Calluses',
 'Cellulite',
 'Chapped Lips',
 'Clarifying',
 'Color Protection',
 'Crows Feet',
 'Damage & Split Ends',
 'Dandruff',
 'Dark Circles',
 'Dark Spots',
 'Depuffing',
 'Dry/Dehydrated Skin',
 'Dry Hair',
 'Dry Scalp',
 'Eczema',
 'Free Radical Damage',
 'Frizz',
 'Hair Growth',
 'Hair Thinning',
 'Heat Damage',
 'Hyperpigmentation',
 'Hypoallergenic',
 'Ingrown Hair',
 'Ingrown Nail',
 'Large Pores',
 'Oily Hair',
 'Oily Skin',
 'Psoriasis',
 'Purifying',
 'Razor Burn & Bumps',
 'Refining',
 'Rosacea & Redness',
 'Scars',
 'Sensitive Skin',
 'Skin Irritation',
 'Sunburn',
 'Sun Protection',
 'Sweat & Body Odor',
 'Wrinkles']

In [9]:
kwp=KeywordProcessor()
kwp.add_keywords_from_list(list_values)
# Extract Key Words
Concern['description'].apply(lambda x: kwp.extract_keywords(x)) #

37723                                  []
37724                             [Frizz]
37725                                  []
37726                                  []
37727                                  []
37728                                  []
37729                                  []
37730                                  []
51136                                  []
51137    [Sensitive Skin, Hypoallergenic]
51138    [Sensitive Skin, Hypoallergenic]
51139                                  []
Name: description, dtype: object

In [39]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [40]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
x=df[df['matches'].astype(str)!='[]']

In [41]:
x

Empty DataFrame
Columns: [product_id, external_id, product_name, long_desc, customer_id, attributes, buckets, values, matches]
Index: []

In [ ]:
x[x['value'].astype(str)!='Greater than 30 Inches'][100:150]

In [ ]:
df[df['external_id'].astype(str)=='00739441']

In [ ]:
# x['matches'].explode().value_counts()

In [ ]:
def split_col(data):
    data['matches']=data['matches'].str.get(0)
    # data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())

In [ ]:
split_col(x)

In [ ]:
len(x['matches'].explode().value_counts())

In [ ]:
# error

In [ ]:
x['matches']=x['matches'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\'",'',str(measure)))
x

In [ ]:
def remove_fractions(numbers):
    if numbers.find('-') > -1:
#         numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

In [ ]:
x['matches']=x['matches'].apply(lambda x: remove_fractions(x))
x

In [ ]:
x['matches']=x['matches'].astype(float)

In [ ]:
x

# matches

In [ ]:
matches=x
matches

In [ ]:
matches['matches']=matches['matches'].astype(float)

In [ ]:
# range function builder

string = range_string.replace(metric, '')
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)

In [ ]:
matches[matches['value'].astype(str)=='Greater than 50 Inches']

In [ ]:
matches['matches']=matches['matches'].astype(float)
matches[curation_col] = matches['matches'].apply(range_app)
matches

In [ ]:
matches['Q:outside_diameter'].explode().value_counts()

# fixed_na

In [ ]:
fixed_na=matches[matches['Q:outside_diameter'].astype(str)=='n/a']

In [ ]:
fixed_na['matches']=fixed_na['matches'].astype(float).apply(lambda x: x-1) #have to add 1 because these were 100.90 and it could not classify it
# na

In [ ]:
fixed_na[curation_col] = fixed_na['matches'].apply(range_app)
fixed_na

In [ ]:
fixed_na['Q:outside_diameter'].explode().value_counts()

# Wipe

In [ ]:
wipe=df[['external_id']].astype(str)
wipe['Q:outside_diameter']=''
wipe

# Null Values

In [ ]:
na_external_id=df[df['matches'].astype(str)=='[]']
na_external_id['external_id']=na_external_id['external_id'].astype(str)

In [ ]:
na_external_id

In [ ]:
error

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [20]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/Motion - {attribute} - match upload_Shawn.csv',index=False)

In [21]:
looks_good('Motion Industries', attribute, df, wipe)